<a href="https://colab.research.google.com/github/mkjubran/Fundamentals-of-AI-and-Machine-Learning/blob/main/NEURAL_NETWORKS_USING_PyTorch_Fully_Connected_Layers_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fitting and Evaluating Multi-layer Perceptron


In this notebook, we will demonstrate how to fit and evaluate a Multi-layer Perceptron (MLP). We will work on a modified version of the cardiovascular dataset from Kaggle (https://www.kaggle.com/code/sulianova/eda-cardiovascular-data/data).

Part of this tutorial is based on the code in this link https://github.com/pytorch/tutorials/blob/master/beginner_source/basics/quickstart_tutorial.py

# Data Preparation

**Clone the dataset Repository**

The prepared dataset after cleaning, removing outliers, and feature engineering can be cloned from the GitHub repository https://github.com/mkjubran/AIData.git as below

In [ ]:
!rm -rf ./AIData
!git clone https://github.com/mkjubran/AIData.git

**Read the dataset**

The data is stored in the cardio_EDA.csv file. Read the input data into a dataframe using the Pandas library (https://pandas.pydata.org/) to read the data.

In [ ]:
import pandas as pd
df = pd.read_csv("/content/AIData/cardio_EDA.csv",sep=";")
df.head()

**Display Data Info**

Display some information about the dataset using the info() method

In [ ]:
df.info()

The dataset contains 53659 records with 15 features for each record. Twelve features are numeric and the rest are objects (strings).

# Clean Data and Remove Outliers

This data has been processed in previous notebooks
- Data Cleaning: https://github.com/mkjubran/Fundamentals-of-AI-and-Machine-Learning/blob/main/EXPLORATORY_DATA_ANALYSIS_%E2%80%93_DATA_CLEANING.ipynb
- Feature Selection and Feature Engineering: https://github.com/mkjubran/Fundamentals-of-AI-and-Machine-Learning/blob/main/EXPLORATORY_DATA_ANALYSIS_%E2%80%93_FEATURE_SELECTION_AND_FEATURE_ENGINEERING.ipynb

As we noticed from the presented sample of the dataset above some features are highly correlated such as the age and the age_year features. So we need to drop one of these features. Besides, we will drop any not needed features such as the 'id' feature.

In [ ]:
df.drop(['id','age'],axis=1, inplace=True)
df.head()

# Encode Categorical Data

We will use hot encoding through the get_dummies() method in pandas to encode the data in the 'gender' and 'smoke' features.

In [ ]:
df = pd.get_dummies(df)
df.head()

Remember to drop one of the columns that resulted from the hot encoding of each feature. Also, make sure that the original features ('age' and 'smoke') are dropped too.

In [ ]:
df.drop(['gender_female','smoke_No'],axis=1,inplace=True)
df.head()

#Split Data for Training and Testing

We will start by specifying the independent variables and the dependent variable. The independent variables are the features that will be used to predict the target feature (class,label). And the dependent variable is the target feature (class, label).

In [ ]:
# independent variables
X=df.drop(['cardio'],axis=1)
X.head()


In [ ]:
# dependet variable (target feature, class, label)
Y=df.cardio
Y.head()

Then we will splitting the dataset into training and testing splits, the split ratio is usually 80% training and 20% testing.

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, random_state=200)
print('Size of the dataset = {}'.format(len(X)))
print('Size of the training dataset = {} ({}%)'.format(len(x_train), 100*len(x_train)/len(X)))
print('Size of the testing dataset = {} ({}%)'.format(len(x_test), 100*len(x_test)/len(X)))

# Training and Testing a Fully Connected Network

Import librarier required for PyTorch.

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

Prepare the data to be processed using PyTorch. We will use DataLoader to manage data. The DataLoader creates batches from data and we don't have to worry about slicing and shuffling data. We will set the batch size to 64.

In [ ]:
class MyDataset():
 
  def __init__(self,X,Y):
    self.x=X
    self.y=Y
 
  def __len__(self):
    return len(self.y)
   
  def __getitem__(self,idx):
    return self.x[idx],self.y[idx]
    

batch_size = 64

Xt_train=torch.tensor(x_train.to_numpy(),dtype=torch.float32)
y_train_tensor=torch.tensor(y_train.to_numpy(),dtype=torch.float32)
Yt_train = y_train_tensor.type(torch.LongTensor)

Xt_test=torch.tensor(x_test.to_numpy(),dtype=torch.float32)
y_test_tensor=torch.tensor(y_test.to_numpy(),dtype=torch.float32)
Yt_test = y_test_tensor.type(torch.LongTensor)

# Create data loaders.
train_dataloader = DataLoader(MyDataset(Xt_train,Yt_train), batch_size=batch_size)
test_dataloader = DataLoader(MyDataset(Xt_test,Yt_test), batch_size=batch_size)

for X, y in test_dataloader:
    print('Shape of X: {}'.format(X.shape))
    print('Shape of y: {} {}'.format(y.shape,y.dtype))
    break

We need to set the deive to run the deep learning. We would llike to have GPUs but the model can run with low speed on CPUs.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print('Using {} device'.format(device))

Creating a model of fully connected layers. To define a neural network in PyTorch, we create a class that inherits from nn.Module.

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(12, 12),
            nn.ReLU(),
            nn.Linear(12, 12),
            nn.ReLU(),
            nn.Linear(12, 10)
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

After defining the model, we need to set the loss function and the optimizer.

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

Training function including the steps of forward pass and backpropagation

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

Testing function to evaluate the model

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
          X, y = X.to(device), y.to(device)

          # Compute prediction error
          pred = model(X)
          test_loss = loss_fn(pred, y).item()
          correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Now, we will train and evaluate the model for 20 epochs.

In [ ]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

# Saving the Models

We wil use PyTorch to save the model

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

# Predict New Values Using Models

Let us use the model to predict the class and compare it with the ground truth

In [ ]:
classes = [0,1]

x = Xt_test[10].to(device)
y = Yt_test[10].to(device)

model.eval()
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print('Predicted: {}, Actual: {}'.format(predicted,actual))